# IN4080: obligatory assignment 1 (Autumn 2024)
 
Mandatory assignment 1 consists of three parts. In Part 1 (6 points), you will test and improve on a BPE (Byte-Pair-Encoding) tokenizer . In Part 2 (7 points), you will estimate an N-gram language model, based on a training corpus and the tokenizer you worked on in Part 1. Finally, in Part 3 (7 points), you will develop a basic classification model to distinguish between Bokmål and Nynorsk sentences.

You should answer all three parts. You are required to get at least 12/20 points to pass. The most important is that you try to answer each question (possibly with some mistakes), to help you gain a better and more concrete understanding of the topics covered during the lectures. There are also bonus questions for those of you who would like to deepen their understanding of the topics covered by this assignment.

- We assume that you have read and are familiar with IFI’s requirements and guidelines for mandatory assignments, see [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-mandatory.html) and [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-guidelines.html).
- This is an individual assignment. You should not deliver joint submissions. 
- You may redeliver in Devilry before the deadline (__Sunday, September 15 at 23:59__), but include all files in the last delivery.
- Only the last delivery will be read! If you deliver more than one file, put them into a zip-archive. You don't have to include in your delivery the files already provided for this assignment. 
- Name your submission _your\_username\_in4080\_mandatory\_1_
- You can work on this assignment either on the IFI machines or on your own computer. 

*The preferred format for the assignment is a completed version of this Jupyter notebook*, containing both your code and explanations about the steps you followed. We want to stress that simply submitting code is __not__ by itself sufficient to complete the assignment - we expect the notebook to also contain explanations of what you have implemented, along with motivations for the choices you made along the way. Preferably use whole sentences, and mathematical formulas if necessary. Explaining in your own words (using concepts we have covered through in the lectures) what you have done and reflecting on your solution is an important part of the learning process - take it seriously!

Regarding the use of LLMs (ChatGPT or similar): you are allowed to use them as 'sparring partner', for instance to clarify something you have not understood. However, you are __not__ allowed to use them to generate solutions (either in part or in full) to the assignment tasks. 

__Technical tip__: Some of the tasks in this assignment will require you to extend methods in classes that are already partly implemented. To implement those methods directly in a Jupyter notebook, you can use the function `setattr` to attach a method to a given class: 

```python
class A:
    pass
a = A()

def foo(self):
    print('hello world!')
    
setattr(A, 'foo', foo)
```

## Part 1 : Tokenisation

----------------
The **tokenisation** is the process of dividing a text into **tokens**, a single unit representing a part of the text. It can be:
   - A whole word (‘cat’).
   - A part of a word (‘gat-’ or ‘-to’).
   - A single character (‘g’, ‘a’, ‘t’, ‘o’).
   - A symbol or punctuation (‘,’, ‘.’, ‘!’).
   - An abstract entity such as a whitespace or special symbols in the pattern.

The goal of tokenization is to obtain an optimal representation of the text, balancing the granularity of the tokens with the ability of the model to understand their meaning. Among the different algorithms used for this process, two of the most common are:

- **Byte-Pair Encoding (BPE)**: This algorithm is based on the frequency with which pairs of characters or subwords appear in the text. The process begins by dividing the text into individual characters, then iteratively combines the most frequent characters or sequences of characters to form larger tokens. This procedure continues until a predetermined number of tokens in the vocabulary is reached, thus allowing an efficient representation for both common words and rare or new terms.

- **Unigram**: Unlike BPE, the Unigram algorithm starts with a large set of possible tokens and progressively eliminates them based on the statistical probability of each token to represent the text well. In this way, the algorithm retains only the tokens that are most useful for understanding the text, achieving a balance between accuracy and vocabulary size.

Tokenization is a fundamental process in Natural Language Processing (NLP) models, as it transforms natural language text into a format that models can process, such as sequences of numbers or vectors. This step is crucial for enabling mathematical models to understand and work with human language. Achieving an optimal balance between the granularity of tokens and their interpretability is key to ensuring that language models perform effectively.

----------------

We will start by building a basic tokenizer relying on white space and punctuation. 

__Task 1.1__ (2 points): Implement the method `split` below such that it takes a text as input and outputs a list of tokens. The tokenisation should simply be done by splitting on white space, except for punctuation markers and other symbols (`.,:;!?-()"`), which should correspond to their own token. For instance, the sentence "Pierre, who works at NR, also teaches at UiO." should be split into 12 tokens.

In [1]:
import re
string = "Pierre, who works at NR, also teaches at UiO."
string_split = re.split(r'(\s+|[.,:;!?()|"/-])', string)
string_split = [t for t in string_split if t.strip()]
print(string_split)
print(len(string_split))

['Pierre', ',', 'who', 'works', 'at', 'NR', ',', 'also', 'teaches', 'at', 'UiO', '.']
12


In [2]:
from typing import List
import re

def basic_tokenize(text: str) -> List[str]:
    """The method should split the text on white space, except for punctuation
    markers that should be considered as tokens of their own (even in the 
    absence of white space before or after their occurrence)"""

    # Implement here your basic tokenisation
    string_split = re.split(r'(\s+|[.,:;!?()|"/-])', text)
    string_split = [t for t in string_split if t.strip()]
    print("Number of tokens are:\t", len(string_split), "\nNumber of types are:\t", len(set(string_split)))
    return string_split

In [3]:
y = basic_tokenize(string)

Number of tokens are:	 12 
Number of types are:	 10


We will now run the tokeniser on a small corpus, the [Norwegian Dependency Treebank](https://www.nb.no/sprakbanken/en/resource-catalogue/oai-nb-no-sbr-10/) (the corpus has been annotated with morphological features, syntactic functions and hierarchical structures, but we'll simply use here the raw text and discard all the annotation layers). We provide you with the data in the files `ndt_train_lm.txt` and `ndt_test_lm.txt`. 

__Task 1.2__ (1 point): Run the tokenizer you have implemented on `ndt_test_lm.txt`. How many tokens were extracted? And how many types (distinct words) were there? 

In [4]:
with open("ndt_test_lm.txt", 'r', encoding="utf-8") as file:
    ndt_test_lm = file.read()

tokens_types = basic_tokenize(ndt_test_lm)

Number of tokens are:	 259236 
Number of types are:	 30050


We shall now use Byte-Pair Encoding (BPE) to limit the vocabulary of the tokenizer to 5,000.  An initial implementation of the algorithm is provided below.

In [5]:
from typing import Dict, List, Tuple, Iterator
import numpy as np
from tqdm.notebook import tqdm

class BPETokenizer:
    """Tokenizer based on the Byte-Pair Encoding algorithm. 
    Note: the current implementation is limited to Latin characters (ISO-8859-1)"""

    def __init__(self, train_corpus_file: str, vocab_size = 5000):
        """Creates a new BPE tokenizer, with merge pairs found using the given
        corpus file. The extraction of merge pairs stops when a vocabulary of 
        size vocab_size is reached."""

        # List of string pairs that should be merged when tokenizing
        # Example: ('e', 't'), which means that 'et' is a possible subword
        # Each string pair is mapped to an unique index number
        # (corresponding to their position in the self.vocab list)
        self.merge_pairs = {}

        # We add as basic vocab all characters of the extended ASCII
        self.vocab = [chr(i) for i in range(256)]

        with open(train_corpus_file) as fd:

            # We first read the corpus, split on white space, and counts the
            # occurrences of each distinct word
            print("Counting word occurrences in corpus %s"%train_corpus_file, end="...", flush=True)
            text = fd.read()
            vocabulary_counts = {}
            for token in text.split():
                vocabulary_counts[token] = vocabulary_counts.get(token, 0) + 1
            print("Done")

            # We then iteratively extend the list of merge pairs until we
            # reach the desired size. Note: to speed up the algorithm, we 
            # extract n merge pairs at each iteration
            progress_bar = tqdm(total=vocab_size)
            while len(self.vocab) < vocab_size:
                most_common_pairs = self.get_most_common_pairs(vocabulary_counts)
                for common_pair in most_common_pairs:
                    self.merge_pairs[common_pair] = len(self.vocab)
                    self.vocab.append("".join(common_pair))
                progress_bar.update(len(most_common_pairs))
        #       print("Examples of new subwords:", ["".join(pair) for pair in most_common_pairs][:10])
            
    def get_most_common_pairs(self, vocabulary_counts: Dict[str,int], 
                              n:int=200) -> List[Tuple[str,str]]:
        """Given a set of distinct words along with their corresponding number 
        of occurrences in the corpus, returns the n most frequent pairs of subwords.       
        """

        # We count the frequencies of consecutive subwords in the vocabulary list
        pair_freqs = {}
        for word, word_count in vocabulary_counts.items():
            subwords = self.tokenize_word(word)
            for i in range(len(subwords)-1):
                byte_pair = (subwords[i], subwords[i+1])
                pair_freqs[byte_pair] = pair_freqs.get(byte_pair, 0) + word_count

        # And return the most frequent ones
        most_freq_pairs = sorted(pair_freqs.keys(), key=lambda x: pair_freqs[x])[::-1][:n]
        return most_freq_pairs

    def __call__(self, input:str, show_progress_bar=True) -> Iterator[str]:
        """Tokenizes a full text"""

        # We first split into whitespace-separated tokens, and then in subwords
        words = input.split()
        for word in tqdm(words) if show_progress_bar else words:
            subwords = self.tokenize_word(word)
            for subword in subwords:
                yield subword
                

    def tokenize_word(self, word):
        """Splits the word into subwords, according to the merge pairs 
        currently stored in self.merge_pairs."""

        # We start with a list of characters
        # (+ a final character to denote the end of the word)    
        splits = list(word) + [" "]

        # We continue until there is nothing left to be merged
        while len(splits)>=2:

            # We extract consecutive subword pairs
            pairs = [(splits[i], splits[i+1]) for i in range(len(splits)-1)]

            # We find the "best" pair of subwords to merge -- that is, the one with the 
            # lowest position in the list of merge rules
            best_pair_to_merge = min(pairs, key=lambda x: self.merge_pairs.get(x, np.inf))
            if best_pair_to_merge in self.merge_pairs:

                # We then merge the two subwords
                for i in range(len(splits)-1):
                    if (splits[i], splits[i+1]) == best_pair_to_merge:
                        merged_subword = self.vocab[self.merge_pairs[best_pair_to_merge]]
                        splits = splits[:i] + [merged_subword] + splits[i+2:]
                        break
            else:
                break
        return splits

__Task 1.3__ (1 point): Learn the BPE tokenizer on the `ndt_train_lm.txt` corpus, and then apply this tokenizer on `ndt_test_lm.txt`. Print the number of tokens and types (distinct subwords) obtained by this tokenizer on the test data. How do those numbers compare to the ones obtained with the basic tokenizer you had implemented earlier? 

In [6]:
with open("ndt_train_lm.txt", 'r', encoding="utf-8") as file:
    ndt_train_lm = file.read()

In [7]:
ndt_training_lm = BPETokenizer(train_corpus_file = 'ndt_train_lm.txt', vocab_size = 5000)

ndt_testing_lm = list(ndt_training_lm(ndt_test_lm))

Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/226096 [00:00<?, ?it/s]

In [8]:
print("Number of tokens are:\t", len(ndt_testing_lm))
print("Number of types are:\t", len(set(ndt_testing_lm)))

Number of tokens are:	 386099
Number of types are:	 4408


**How do those numbers compare to the ones obtained with the basic tokenizer you had implemented earlier ?**
|  | basic_tokenize | BPETokenizer |
|-----------|-----------|-----------|
| Number of tokens | 259236  | 386099 |
| Number of types | 30050 | 4408 |


The number of tokens and type are different because of the byte pair coding, because I just implement a tokenization at character level, so I only split words.

In [9]:
print("vocab dim:\t", len(ndt_training_lm.vocab))

vocab dim:	 5056


__Task 1.4__ (2 points): The current BPE implementation is that it treats all characters in the same manner. A rather inconvenient side effect is that letters may be merged together with punctuation markers (like 'ing', ',' --> 'ing,'), if they are not separated by white space. Modify the implementation of the BPE algorithm above to prevent punctuation markers to be merged with letters. 

In [10]:
import re

class BPETokenizer2:
    """Tokenizer based on the Byte-Pair Encoding algorithm, extended to handle punctuation splitting.
    Note: the current implementation handles Latin characters (ISO-8859-1) and punctuation."""

    def __init__(self, train_corpus_file: str, vocab_size=5000):
        """Creates a new BPE tokenizer, with merge pairs found using the given
        corpus file. The extraction of merge pairs stops when a vocabulary of 
        size vocab_size is reached."""
        self.merge_pairs = {}
        self.vocab = [chr(i) for i in range(256)] + list('.,!?;:"\'()[]{}')

        with open(train_corpus_file) as fd:
            print("Counting word occurrences in corpus %s" % train_corpus_file, end="...", flush=True)
            text = fd.read()
            vocabulary_counts = {}
            for token in self.split_on_punctuation(text):
                vocabulary_counts[token] = vocabulary_counts.get(token, 0) + 1
            print("Done")

            progress_bar = tqdm(total=vocab_size)
            while len(self.vocab) < vocab_size:
                most_common_pairs = self.get_most_common_pairs(vocabulary_counts)
                for common_pair in most_common_pairs:
                    self.merge_pairs[common_pair] = len(self.vocab)
                    self.vocab.append("".join(common_pair))
                progress_bar.update(len(most_common_pairs))

    def split_on_punctuation(self, text: str) -> List[str]:
        """Splits text into words and punctuation marks"""
        return re.findall(r"\w+|[.,!?;:]", text)

    def get_most_common_pairs(self, vocabulary_counts: Dict[str, int], n: int = 200) -> List[Tuple[str, str]]:
        """Returns the n most frequent pairs of subwords."""
        pair_freqs = {}
        for word, word_count in vocabulary_counts.items():
            subwords = self.tokenize_word(word)
            for i in range(len(subwords) - 1):
                byte_pair = (subwords[i], subwords[i + 1])
                pair_freqs[byte_pair] = pair_freqs.get(byte_pair, 0) + word_count
        most_freq_pairs = sorted(pair_freqs.keys(), key=lambda x: pair_freqs[x])[::-1][:n]
        return most_freq_pairs

    def __call__(self, input: str, show_progress_bar=True) -> Iterator[str]:
        """Tokenizes a full text."""
        words = self.split_on_punctuation(input)
        for word in tqdm(words) if show_progress_bar else words:
            subwords = self.tokenize_word(word)
            for subword in subwords:
                yield subword

    def tokenize_word(self, word: str) -> List[str]:
        """Splits the word into subwords, according to the merge pairs."""
        splits = list(word) + [" "]
        while len(splits) >= 2:
            pairs = [(splits[i], splits[i + 1]) for i in range(len(splits) - 1)]
            best_pair_to_merge = min(pairs, key=lambda x: self.merge_pairs.get(x, np.inf))
            if best_pair_to_merge in self.merge_pairs:
                for i in range(len(splits) - 1):
                    if (splits[i], splits[i + 1]) == best_pair_to_merge:
                        merged_subword = self.vocab[self.merge_pairs[best_pair_to_merge]]
                        splits = splits[:i] + [merged_subword] + splits[i + 2:]
                        break
            else:
                break
        return splits

In [11]:
ndt_training_lm2 = BPETokenizer2(train_corpus_file = 'ndt_train_lm.txt', vocab_size = 5000)

ndt_testing_lm2 = list(ndt_training_lm2(ndt_test_lm))

Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/251297 [00:00<?, ?it/s]

In [12]:
print("Number of tokens are:\t", len(ndt_testing_lm2))
print("Number of types are:\t", len(set(ndt_testing_lm2)))

Number of tokens are:	 383605
Number of types are:	 4306


In [13]:
print("vocab dim:\t", len(ndt_training_lm2.vocab))

vocab dim:	 5070


__Task 1.5__ (_optional, 2 extra points_): In a [tweet](https://x.com/karpathy/status/1759996551378940395) published earlier this year, the well-known AI researcher Andrej Karpathy stressed that many of the current limitations of Large Language Models are actually a product of the tokenisation step. Explain at least 4 of the problems he mentioned in his tweet (you can of course search online, or watch Karpathy's own video lecture on tokenization).

1. **Why can't LLM do super simple string processing tasks like reversing a string?**
Tokenisation affects the way LLM sees text. Since text is divided into tokens rather than characters, inverting a string, which seems simple for humans, can become complex for an LLM working with tokenised units.

2. **Why is LLM bad at simple arithmetic?**
Simple arithmetic can be problematic because numbers might be tokenized in a way that doesn’t preserve their numerical meaning. For example, breaking down a number like "12345" into multiple tokens can confuse the model during mathematical operations.

3. **Why did GPT-2 have more than necessary trouble coding in Python?** 
In programming languages, even small changes in tokens (like removing a whitespace or breaking a string into unexpected parts) can lead to significant errors. This also applies to GPT models like GPT-2 that tokenize code ineffectively, causing trouble in generating accurate or functional code.

4. **What is this weird warning I get about a "trailing whitespace"?**
Tokenization can treat whitespace characters as significant, so trailing whitespace might generate errors or warnings. This is particularly problematic in environments like code generation, where whitespace can be crucial.

## Part 2: N-gram Language Models

-----
An **N-gram** language model is a probabilistic model used to predict the next word in a sequence of text based on the previous $N-1$ words. N-grams are sequences of $N$ consecutive elements (usually words or characters) that appear in a text.

The main goal of an N-gram language model is to estimate the conditional probability of a word given a sequence of preceding words. Mathematically, the problem of modeling a sequence of words $w_1, w_2, ..., w_n$ is based on calculating the joint probability:

$$
P(w_1, ..., w_n) = \prod_{k=1}^n P(w_k \mid w_1, w_2, ..., w_{k-1})
$$

However, this approach becomes computationally impractical for long sequences, as the amount of data required to accurately estimate these probabilities grows exponentially with the length of the sequence.

### The Markov assumption
To solve this problem, a **Markov assumption** is used: it is assumed that the probability of a word depends only on the previous $N-1$ words. In other words, instead of considering the entire preceding sequence of words, only the last $N-1$ words are taken into account. The N-gram model thus approximates the probability of a word $w_i$ as:

$$
P(w_i | w_1, ..., w_{i-1}) \approx P(w_i | w_{i-N+1}, ..., w_{i-1})
$$

Depending on the value of $N$, the model has different names:
- **Unigram** ($N=1$): The probability of each word is independent of the others. The probability of a sequence is simply the product of the probabilities of each word:
  
  $$
  P(w_i \mid w_1, ..., w_{i-1}) \approx P(w_i)
  $$

- **Bigram** ($N=2$): The probability of each word depends only on the previous word:

  $$
  P(w_i \mid w_1, ..., w_{i-1}) \approx P(w_i | w_{i-1})
  $$

- **Trigram** ($N=3$): The probability of each word depends on the two previous words:

  $$
  P(w_i \mid w_1, ..., w_{i-1}) \approx P(w_i | w_{i-2}, w_{i-1})
  $$

### Probability Estimation
To calculate probabilities in an N-gram model, the frequencies of N-grams observed in the training corpus are used. The conditional probability of an N-gram can be estimated as:

$$
P(w_i | w_{i-N+1}, ..., w_{i-1}) = \frac{C(w_{i-N+1}, ..., w_{i-1}, w_i)}{C(w_{i-N+1}, ..., w_{i-1})}
$$

where:
- $C(w_{i-N+1}, ..., w_{i-1}, w_i)$ is the number of times the sequence of N words appears in the corpus.
- $C(w_{i-N+1}, ..., w_{i-1})$ is the number of times the first $N-1$ words of the sequence appear in the corpus.

### Smoothing
One of the main problems with N-gram models is **data sparsity**: many possible N-grams may never appear or appear very rarely in the training corpus, making it difficult to estimate their probabilities. To address this issue, **smoothing** techniques are used, such as:

- **Laplace Smoothing**: Adds one to each count to avoid zero probabilities:

  $$
  P(w_i | w_{i-N+1}, ..., w_{i-1}) = \frac{C(w_{i-N+1}, ..., w_{i-1}, w_i) \textcolor{cyan}{+ 1}}{C(w_{i-N+1}, ..., w_{i-1}) \textcolor{cyan}{+ 1 \cdot V}}
  $$

  where $V$ is the total number of tokens in the vocabulary.

--------

We will now train simple N-gram language models on the NDT corpus, using the tokenizers we have developed in Part 1.

Here is the skeleton of the code:

In [14]:
import numpy as np
from abc import abstractmethod

class LanguageModel:
    """Generic class for running operations on language models, using a BPE tokenizer"""

    def __init__(self, tokenizer: BPETokenizer):
        """Build an abstract language model using the provided tokenizer"""
        self.tokenizer = tokenizer

    @abstractmethod
    def predict(self, context_tokens: List[str]):
        """Given a list of context tokens (=previous tokens), returns a dictionary
           mapping each possible token to its probability"""
        raise NotImplementedError()
    
    @abstractmethod
    def get_perplexity(self, text: str):
        """Computes the perplexity of the given text according to the LM"""

        print("Tokenising input text:")
        tokens = list(self.tokenizer(text))
        
        print("Computing perplexity:")
        log_probs = 0
        for i in tqdm(range(len(tokens))):
            context_tokens = ["<s>"] + tokens[:i]
            predict_distrib = self.predict(context_tokens)

            # We add the log-probabilities
            log_probs += np.log(predict_distrib[tokens[i]])
            
        perplexity = np.exp(-log_probs/len(tokens))
        return perplexity

class NGramLanguageModel(LanguageModel):
    """Representation of a N-gram-based language model"""

    def __init__(self, training_corpus_file: str, tokenizer:BPETokenizer, ngram_size:int=3,
                 alpha_smoothing:float=1):
        """Initialize the N-gram model with:
        - a file path to a training corpus to estimate the N-gram probabilities
        - an already learned BPE tokenizer
        - an N-gram size
        - a smoothing parameter (Laplace smoothing)"""
        
        LanguageModel.__init__(self, tokenizer)
        self.ngram_size = ngram_size
        
        # We define a simple backoff distribution (here just a uniform distribution)
        self.default_distrib = {token:1/len(tokenizer.vocab) for token in tokenizer.vocab}

        # Dictionary mapping a context (for instance the two preceding words if ngram_size=3)
        # to another dictionary specifying the probability of each possible word in the 
        # vocabulary. The context should be a tuple of tokens.
        self.ngram_probs = {}
        with open(training_corpus_file) as fd:   

            # based on the training corpus, tokenizer, ngram-size and smoothing parameter,
            # fill the self.ngram_probs with the correct N-gram probabilities  
            raise NotImplementedError()


    def predict(self, context_tokens: List[str]):
        """Given a list of preceding tokens, returns the probability distribution 
        over the next possible token."""

        # We restrict the contextual tokens to (N-1) tokens
        context_tokens = tuple(context_tokens[-self.ngram_size+1:])

        # If the contextual tokens were indeed observed in the corpus, simply
        # returns the precomputed probabilities
        if context_tokens in self.ngram_probs:
            return self.ngram_probs[context_tokens]
        
        # Otherwise, we return a uniform distribution over possible tokens
        else:
            return self.default_distrib

__Task 2.1__ (6 points): Complete the initialization method `__init__` to estimate the correct N-gram probabilities (with smoothing) based on the corpus. Don't worry about making your implementation super-efficient (although you can if you wish).

In [15]:
from collections import defaultdict, Counter

def __init__(self, training_corpus_file: str, tokenizer: BPETokenizer, ngram_size: int = 2, alpha_smoothing: float = 0.1):
    """Initialize the N-gram model with:
    - a file path to a training corpus to estimate the N-gram probabilities
    - an already learned BPE tokenizer
    - an N-gram size
    - a smoothing parameter (Laplace smoothing)
    """
    LanguageModel.__init__(self, tokenizer)
    self.ngram_size = ngram_size

    # Define a simple backoff distribution (uniform distribution)
    self.default_distrib = {token: 1/len(tokenizer.vocab) for token in tokenizer.vocab}

    # Dictionary mapping contexts (tuples of preceding words) to probability distributions
    self.ngram_probs = defaultdict(Counter)
    context_counts = defaultdict(Counter)
    
    # Read the corpus and tokenize it
    with open(training_corpus_file, 'r') as fd:
        for line in fd:
            tokens = ["<s>"] + list(self.tokenizer(line.strip())) + ["</s>"]
            
            # Create n-grams and context for each token
            for i in range(len(tokens) - ngram_size + 1):
                context = tuple(tokens[i:i + ngram_size - 1])
                token = tokens[i + ngram_size - 1]
                context_counts[context][token] += 1
    
    # Calculate smoothed probabilities
    vocab_size = len(self.tokenizer.vocab)
    
    for context, token_counts in context_counts.items():
        total_count = sum(token_counts.values()) + alpha_smoothing * vocab_size
        for token, count in token_counts.items():
            self.ngram_probs[context][token] = (count + alpha_smoothing) / total_count
        
        for token in self.tokenizer.vocab:
            if token not in self.ngram_probs[context]:
                self.ngram_probs[context][token] = alpha_smoothing / total_count

setattr(NGramLanguageModel, '__init__', __init__)

-------------------
### Evaluation of Language Models (Perplexity)

**Perplexity** is a metric used to evaluate language models, representing how well a model predicts a sequence of words. It is defined as the inverse probability of a sentence, normalized by the number of words:

$$
PPL(w_1, \dots, w_n) = \sqrt[n]{\frac{1}{P(w_1, \dots, w_n)}}
$$

- Minimizing perplexity is equivalent to maximizing the probability of the word sequence.
- The probability of a sequence lies within the range $[0, 1]$, while perplexity ranges from $1$ to $\infty$.

When applying the **bigram Markov assumption**, perplexity is calculated as follows:

$$
PPL(w_1, \dots, w_n) = \sqrt[n]{\prod_{i=1}^{n} \frac{1}{P(w_i \mid w_{i-1})}}
$$

In this case, the model assumes that the probability of each word depends only on the previous word in the sequence.

-------------------

__Task 2.2__ (1 point): Train your language model in `ndt_train_lm.txt`, and compute its perplexity on the test data in `ndt_test_lm.txt`. The perplexity can be computed by calling the method `get_perplexity`. <br>
(_Note_: if the training takes too much time, feel free to stop the process after looking at a fraction of the corpus, at least while you are testing/developing your training setup).

In [16]:
model = NGramLanguageModel(training_corpus_file='ndt_train_lm.txt', tokenizer=BPETokenizer(train_corpus_file= 'ndt_train_lm.txt', vocab_size = 5000))

Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/872824 [00:00<?, ?it/s]

In [17]:
perplexity = model.get_perplexity(ndt_test_lm)
print(f"Test Perplexity: {perplexity}")

Tokenising input text:


  0%|          | 0/226096 [00:00<?, ?it/s]

Computing perplexity:


  0%|          | 0/386099 [00:00<?, ?it/s]

Test Perplexity: 193.0345419254361


|  Perplexity| 193.0345419254361
|-----------|-----------|

__Task 2.3__ (_optional_, 4 bonus points): Improve the language model you have just developed. You can choose to focus on improving your model through a backoff mechanism, interpolation, or both. Once you are done, compute the perplexity again on the test corpus to ensure the language model has indeed improved.

## Part 3: Text classification

We will finally use the texts from the Norwegian Dependency Treebank for a classification task -- more precisely to determine whether a sentence is likely to be written in Bokmål or Nynorsk. To this end, we will use a simple bag-of-word setup (or more precisely a bag-of-_subwords_, since we will rely on the subwords extracted using BPE) along with a logistic regression model. As there is only two, mutually exclusive classes, you can view the task as a binary classification problem. 

The training data is found in `ndt_train_class.txt` and simply consists of a list of sentences, each sentence being mapped to a language form (Bokmål: `nob` or Nynorsk: `nno`). The language form is written at the end of each line, separated by a `\t`. Note the training examples are currently _not_ shuffled.

To train and apply your classifier, the easiest is to use the [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model from `scikit-learn`.

__Task 3.1__ (2 points): Create a `N x V` matrix in which each line corresponds to a training example (out of `N` training instances) and each row corresponds to an individual feature, in this case the presence/absence of a particular subword in the sentence. In other words, there should be a total of `V` features, where `V` is the total size of the vocabulary for our BPE tokenizer. Also create a vector of length `N` with a value of `1` if the sentence was marked as Nynorsk, and 0 if is was marked as Bokmål. 

In [18]:
with open("ndt_train_class.txt", 'r', encoding="utf-8") as file:
    ndt_train_class = file.read()

ndt_training_class = BPETokenizer2(train_corpus_file = 'ndt_train_class.txt', vocab_size = 5000)

Counting word occurrences in corpus ndt_train_class.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

In [19]:
# I defined a function to create the design matrix. The function takes as input the text, the method and the vocabulary to use.

def design_matrix_creation(a_text, a_method, vocabulary):
    vocab_index = {word: idx for idx, word in enumerate(vocabulary)}
    a_text_split = re.split(r"\n", a_text)
    FMatrix = np.zeros((len(a_text_split), len(vocabulary)))
    f = np.zeros(len(a_text_split))

    for i, sentence in enumerate(a_text_split[:-1]):
        t = re.split(r"\t", sentence)
        tokenization = list(a_method(t[0], show_progress_bar=False))

        for token in tokenization:
            if token in vocab_index:
                position = vocab_index[token]
                FMatrix[i][position] = 1

        if t[1] == 'nno':
            f[i] = 1
    return FMatrix, f

In [20]:
vocabulary = ndt_training_class.vocab

In [21]:
F_train_Matrix, f_train = design_matrix_creation(ndt_train_class, ndt_training_class, vocabulary)
print(f'The design matrix is:\n{F_train_Matrix}\n\n', f'The vector is:\n {f_train}')

The design matrix is:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

 The vector is:
 [1. 1. 1. ... 0. 0. 0.]


__Task 3.2__ (2 points): Use the data matrix you have just filled to train a logistic regression model (see the documentation on [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) for more details). We recommend to use the `liblinear` solver. 

In [22]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver = 'liblinear')

model.fit(F_train_Matrix, f_train)

train_predictions = model.predict(F_train_Matrix)

__Task 3.3__ (1 point): Now apply the learned logistic regression model to the test set in `ndt_test_class.txt`, and evaluate its performance in terms of accuracy, recall and precision (you can use the functionalities in `sklearn.metrics` to compute those easily).

In [23]:
with open("ndt_test_class.txt", 'r', encoding="utf-8") as file:
    ndt_test_class = file.read()

ndt_testing_class = list(ndt_training_class(ndt_test_class))

  0%|          | 0/608473 [00:00<?, ?it/s]

In [24]:
F_test_Matrix, f_test = design_matrix_creation(ndt_test_class, ndt_training_class, vocabulary)
print(f'The design matrix is:\n{F_test_Matrix}\n\n', f'The vector is:\n {f_test}')

The design matrix is:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

 The vector is:
 [1. 1. 1. ... 0. 0. 0.]


In [25]:
test_predictions = model.predict(F_test_Matrix)

In [26]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

accuracy = accuracy_score(f_test, test_predictions)
recall = recall_score(f_test, test_predictions)
precision = precision_score(f_test, test_predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")

Accuracy: 0.9745
Recall: 0.9627
Precision: 0.9855


__Task 3.4__ (2 points): Inspect the weights learned by your logistic regression model (in `coef_`) and find the 5 subwords that contribute _the most_ to the classification of the sentence in Nynorsk. Also find the 5 subwords that contribute the most to the classification of the sentence in Bokmål. Do those weights make sense, according to what you know about Bokmål and Nynorsk ? 

In [27]:
weights = model.coef_[0]

weight_subword_map = {subword: weight for subword, weight in zip(vocabulary, weights)}

top_nynorsk_subwords = sorted(weight_subword_map.items(), key=lambda x: x[1], reverse=True)[:5]
top_bokmaal_subwords = sorted(weight_subword_map.items(), key=lambda x: x[1], reverse=True)[-5:]

print("Top 5 subwords for Nynorsk:")
for subword, weight in top_nynorsk_subwords:
    print(f"Subword: {subword}, Weight: {weight:.4f}")

print("\nTop 5 subwords for Bokmål:")
for subword, weight in top_bokmaal_subwords:
    print(f"Subword: {subword}, Weight: {weight:.4f}")

Top 5 subwords for Nynorsk:
Subword: ikkje , Weight: 4.6689
Subword: frå , Weight: 4.2924
Subword: eit , Weight: 4.0350
Subword: dei , Weight: 3.6508
Subword: ein , Weight: 3.3407

Top 5 subwords for Bokmål:
Subword: ble , Weight: -2.7519
Subword: ikke , Weight: -3.1778
Subword: fra , Weight: -3.3179
Subword: sier , Weight: -3.3302
Subword: jeg , Weight: -3.3816
